# ResNet (Deep Residual Learning) の実装

このノートブックでは、2015年のImageNet LSVRCで圧倒的な性能を示したResNet (Residual Network) のアーキテクチャについて学び、その中核となる**残差学習 (Residual Learning)** と**スキップ接続 (Shortcut Connection)** のアイデアを理解します。
NumPyで主要な概念を実装・確認した後、PyTorchを使ってCIFAR-10データセット用に調整したResNet風モデルを実装し、学習と評価を行います。

**参考論文:**
*   He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep residual learning for image recognition. In *Proceedings of the IEEE conference on computer vision and pattern recognition* (pp. 770-778).

**このノートブックで学ぶこと:**
1.  ResNetの設計思想（Degradation Problemと残差学習）
2.  主要なコンポーネントの理解とNumPyによる概念実装:
    *   スキップ接続（Identity Mapping と Projection Shortcut）
    *   残差ブロック（Basic Block と Bottleneck Block）
    *   バッチ正規化 (Batch Normalization) の概念と簡易実装
3.  PyTorchを使ったResNet風モデル（CIFAR-10用）の実装
4.  CIFAR-10データセットでの学習と評価

**前提知識:**
*   GoogLeNetのノートブックで学んだCNNの基礎とPyTorchによる実装経験
*   バッチ正規化の基本的な概念（もし未習得ならここで補足します）


## 1. 必要なライブラリのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

print(f"PyTorch Version: {torch.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

PyTorch Version: 2.5.0+cu124
Torchvision Version: 0.20.0+cu124
Using device: cuda


## 2. ResNetの主要な設計思想とNumPyによる概念実装

ResNetは、従来の非常に深いネットワークが学習しにくいという「Degradation Problem」（劣化問題）を解決するために提案されました。


### 2.1 Degradation Problem と 残差学習 (Residual Learning)

*   **Degradation Problem:**
    理論的には、より深いネットワークは浅いネットワーク以上の性能を発揮できるはずです（浅いネットワークの層をコピーし、追加の層を恒等写像として学習すれば同等の性能になるため）。しかし、実際には、ある程度以上層を深くすると、訓練誤差もテスト誤差も悪化してしまう現象が観測されました。これは過学習ではなく、深いネットワークの最適化が困難であることに起因します。

*   **残差学習のアイデア:**
    ネットワークの層に、目的の写像 $H(x)$ を直接学習させる代わりに、入力 $x$ との差分である**残差関数** $F(x) = H(x) - x$ を学習させます。そして、その層（またはブロック）の出力は $F(x) + x$ となります。
    この $x$ を出力に加える部分が**スキップ接続**または**ショートカット接続**と呼ばれます。

    **なぜ残差学習が有効か？**
    もし理想的な写像が恒等写像 $H(x) = x$ に近い場合、残差 $F(x)$ をゼロに近づけるように学習する方が、恒等写像そのものを複数の非線形層で近似するよりも容易であると考えられます。スキップ接続は勾配の流れを良くし、深いネットワークの学習を助けます。

### 2.2 スキップ接続 (Shortcut Connection) と 残差ブロック (Residual Block)

*   **Identity Shortcut (恒等ショートカット):**
    入力 $x$ と残差ブロックの出力 $F(x)$ の次元が同じ場合、単純に $F(x) + x$ を計算します。

*   **Projection Shortcut (射影ショートカット):**
    $F(x)$ のチャネル数や空間的次元が入力 $x$ と異なる場合（例: プーリングやストライド付き畳み込みで次元が変化した後）、$x$ の次元を $F(x)$ に合わせるために、1x1畳み込みなどを用いた線形射影 $W_s x$ を行い、$F(x) + W_s x$ を計算します。

*   **残差ブロックの種類 (論文 Figure 3, 5):**
    *   **Basic Block:** 主にResNet-18やResNet-34のような比較的浅いモデルで使用されます。2つの3x3畳み込み層から構成されます。
        `x -> Conv3x3 -> BN -> ReLU -> Conv3x3 -> BN -> (+ x) -> ReLU`
    *   **Bottleneck Block:** 主にResNet-50以上の深いモデルで使用され、計算効率を高めます。
        `x -> Conv1x1 (次元削減) -> BN -> ReLU -> Conv3x3 -> BN -> ReLU -> Conv1x1 (次元復元) -> BN -> (+ x) -> ReLU`


#### 2.2.1 NumPyによるスキップ接続の概念確認

In [2]:
print("--- スキップ接続の概念確認 ---")
# 入力x (例: 1チャネル、2x2の特徴マップ)
x_skip = np.array([[[1,2],[3,4]]], dtype=np.float32)
print("入力 x:\n", x_skip)

# 残差ブロックF(x)の出力 (仮)
F_x_output = np.array([[[0.1, -0.2],[0.3, 0.05]]], dtype=np.float32)
print("\n残差ブロック F(x)の出力:\n", F_x_output)

# Identity Shortcut
output_identity = F_x_output + x_skip
print("\nF(x) + x (Identity Shortcut):\n", output_identity)

# Projection Shortcut (チャネル数を増やす例)
# xが1チャネル、F(x)が2チャネルだと仮定。xを2チャネルに射影する。
# 簡単のため、xを2倍して2チャネルにするような射影Wsを考える (実際は学習可能な1x1 conv)
x_skip_proj_channel = np.concatenate((x_skip * 0.5, x_skip * 0.3), axis=0) # (2, 2, 2)
F_x_output_2channel = np.random.rand(2,2,2).astype(np.float32) * 0.1
print("\n射影されたx (Ws x) (2チャネル):\n", x_skip_proj_channel)
print("\n残差ブロック F(x)の出力 (2チャネル):\n", F_x_output_2channel)

output_projection = F_x_output_2channel + x_skip_proj_channel
print("\nF(x) + Ws x (Projection Shortcut):\n", output_projection)

--- スキップ接続の概念確認 ---
入力 x:
 [[[1. 2.]
  [3. 4.]]]

残差ブロック F(x)の出力:
 [[[ 0.1  -0.2 ]
  [ 0.3   0.05]]]

F(x) + x (Identity Shortcut):
 [[[1.1  1.8 ]
  [3.3  4.05]]]

射影されたx (Ws x) (2チャネル):
 [[[0.5        1.        ]
  [1.5        2.        ]]

 [[0.3        0.6       ]
  [0.90000004 1.2       ]]]

残差ブロック F(x)の出力 (2チャネル):
 [[[0.07360736 0.08230535]
  [0.09459218 0.08022221]]

 [[0.03699499 0.00854771]
  [0.02379257 0.03262518]]]

F(x) + Ws x (Projection Shortcut):
 [[[0.5736073  1.0823053 ]
  [1.5945922  2.0802221 ]]

 [[0.336995   0.60854775]
  [0.9237926  1.2326252 ]]]


## 2.3 バッチ正規化 (Batch Normalization - BN)

*   **概念:**
    バッチ正規化は、ミニバッチ内の各特徴量（チャネル）に対して、平均が0、分散が1になるように正規化する手法です。その後、学習可能なスケールパラメータ $\gamma$ とシフトパラメータ $\beta$ を用いてアフィン変換を行います。
    $ \hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} $
    $ y_i = \gamma \hat{x}_i + \beta $
    ここで、$\mu_B$ と $\sigma_B^2$ は現在のミニバッチにおける特徴量の平均と分散、$\epsilon$ はゼロ除算を防ぐための微小値です。

*   **利点:**
    *   **内部共変量シフト (Internal Covariate Shift) の低減:** 各層の入力分布が学習中に変化する現象を抑制し、学習を安定化させます。
    *   **学習率を高く設定可能に:** 学習が安定するため、より大きな学習率を使え、学習が高速化します。
    *   **初期値への依存性の低減。**
    *   **正則化効果:** ミニバッチ統計量を使うため、一種のノイズが加わり、弱い正則化効果があると言われています。Dropoutの必要性を減らすことも。

*   **ResNetでの使われ方:**
    ResNetでは、各畳み込み層の後、活性化関数(ReLU)の**前**にバッチ正規化を適用するのが一般的です。

*   **NumPyによる概念実装 (順伝播 - 訓練時):**
    （推論時は、訓練全体で計算された移動平均と移動分散を使用します）

In [3]:
def batch_norm_forward_numpy(X_bn, gamma_bn, beta_bn, epsilon=1e-5, training_mode=True, running_mean=None, running_var=None):
    """
    バッチ正規化の順伝播 (NumPy実装 - 訓練時と推論時の一部)
    入力X_bnは (N, C, H, W) または (N, D) を想定。正規化はチャネル/特徴量ごと。
    簡単のため、ここでは (N, D) の2D入力を想定。CNNの場合はチャネルごと。
    Args:
        X_bn: 入力データ (N, D)
        gamma_bn: スケールパラメータ (D,)
        beta_bn: シフトパラメータ (D,)
        epsilon: ゼロ除算防止の微小値
        training_mode: 訓練モードか推論モードか
        running_mean: 推論時に使用する移動平均
        running_var: 推論時に使用する移動分散
    Returns:
        out: 正規化後の出力
        cache: (訓練時のみ) 逆伝播で使う値
    """
    if training_mode:
        # ミニバッチの平均と分散を計算 (特徴量ごと)
        # axis=0 はサンプル方向の平均/分散
        batch_mean = np.mean(X_bn, axis=0) 
        batch_var = np.var(X_bn, axis=0)
        
        # 正規化
        X_normalized = (X_bn - batch_mean) / np.sqrt(batch_var + epsilon)
        
        # スケールとシフト
        out = gamma_bn * X_normalized + beta_bn
        
        # 訓練時は、推論で使うための移動平均と移動分散も更新する (ここでは省略)
        cache = (X_bn, X_normalized, batch_mean, batch_var, gamma_bn, epsilon)
        return out, cache
    else: # 推論モード
        if running_mean is None or running_var is None:
            raise ValueError("Running mean and variance must be provided in inference mode.")
        X_normalized = (X_bn - running_mean) / np.sqrt(running_var + epsilon)
        out = gamma_bn * X_normalized + beta_bn
        return out, None


print("\n--- バッチ正規化テスト ---")
# (N=3サンプル, D=2特徴量)
X_test_bn = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=np.float32)
gamma_test_bn = np.array([1.0, 1.5], dtype=np.float32) # 特徴量ごとのスケール
beta_test_bn = np.array([0.0, 0.5], dtype=np.float32)  # 特徴量ごとのシフト

# 訓練モード
out_bn_train, cache_bn = batch_norm_forward_numpy(X_test_bn, gamma_test_bn, beta_test_bn)
print("BN入力:\n", X_test_bn)
print("BN出力 (訓練時):\n", out_bn_train)
# 特徴量0の平均は3, 分散は ((1-3)^2+(3-3)^2+(5-3)^2)/3 = (4+0+4)/3 = 8/3
# 特徴量1の平均は4, 分散は ((2-4)^2+(4-4)^2+(6-4)^2)/3 = (4+0+4)/3 = 8/3
# 正規化後の平均は約0、分散は約1になるはず (スケール・シフト前)

# 推論モード (訓練時の統計を使うと仮定)
# running_mean_test = cache_bn[2]
# running_var_test = cache_bn[3]
# out_bn_eval, _ = batch_norm_forward_numpy(X_test_bn, gamma_test_bn, beta_test_bn, 
#                                         training_mode=False, 
#                                         running_mean=running_mean_test, 
#                                         running_var=running_var_test)
# print("BN出力 (推論時、訓練時の統計使用):\n", out_bn_eval) # 訓練時と同じになるはず


--- バッチ正規化テスト ---
BN入力:
 [[1. 2.]
 [3. 4.]
 [5. 6.]]
BN出力 (訓練時):
 [[-1.2247427 -1.337114 ]
 [ 0.         0.5      ]
 [ 1.2247427  2.3371139]]


## 3. ResNet アーキテクチャの概要

ResNetには様々な深さのバリエーションがあります (ResNet-18, 34, 50, 101, 152など)。
基本的な構造は以下の通りです。

1.  **初期畳み込み層:** 通常、大きめのカーネル（例: 7x7）とストライド2の畳み込み層、その後にマックスプーリング。
2.  **残差ブロックのスタック:** 複数の残差ブロック（Basic BlockまたはBottleneck Block）を積み重ねます。
    *   特徴マップの空間的次元を半分にする際（ダウンサンプリング）は、通常、そのステージの最初の残差ブロック内の畳み込み層のストライドを2にするか、プーリング層を挟みます。同時にチャネル数を2倍にすることが多いです。
    *   ショートカット接続も、次元が変化する場合はProjection Shortcut（通常は1x1畳み込み）を使用します。
3.  **Global Average Pooling (GAP):** 最後の残差ブロックの出力に対して適用。
4.  **全結合層:** GAPの出力を受けて最終的なクラス分類を行う（通常1層）。
5.  **Softmax:** 出力層。

**CIFAR-10用の調整:**
CIFAR-10の入力画像サイズ (32x32) はImageNet (224x224) よりずっと小さいため、
*   初期の7x7畳み込みとマックスプーリングは不要か、より小さなカーネル/ストライド（例: 3x3畳み込み、ストライド1、パディング1、マックスプーリングなし）に置き換えます。
*   ダウンサンプリングの回数も減らします（通常2-3回）。
*   残差ブロック内のチャネル数もImageNet用より小さくします。

In [ ]:
# CIFAR-10 データセットの準備 (VGGノートブックと同様)
transform_cifar_resnet = transforms.Compose([
    transforms.RandomCrop(32, padding=4), # Data Augmentation
    transforms.RandomHorizontalFlip(),    # Data Augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
transform_cifar_test_resnet = transforms.Compose([ # テスト時はAugmentationなし
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


train_dataset_cifar_res = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                       download=True, transform=transform_cifar_resnet)
test_dataset_cifar_res = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                      download=True, transform=transform_cifar_test_resnet)

batch_size_cifar_res = 128 
train_loader_cifar_res = DataLoader(train_dataset_cifar_res, batch_size=batch_size_cifar_res, shuffle=True, num_workers=2)
test_loader_cifar_res = DataLoader(test_dataset_cifar_res, batch_size=batch_size_cifar_res, shuffle=False, num_workers=2)

classes_cifar_res = ('plane', 'car', 'bird', 'cat', 'deer', 
                     'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# ResNetのBasicBlock
class BasicBlock(nn.Module):
    expansion = 1 # BasicBlockでは出力チャネル数は入力チャネル数と同じ (expansion=1)

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        # self.relu = nn.ReLU(inplace=True) # ここではReLUをforwardで適用

        self.shortcut = nn.Sequential()
        # 入力と出力のチャネル数またはストライドが異なる場合、ショートカットで次元を調整
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x) # ショートカット接続の準備

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        out += identity # 残差を加算
        out = F.relu(out) # 最後にReLU
        return out

In [ ]:
# ResNetのBottleneckBlock (ResNet-50以上で使われる)
class BottleneckBlock(nn.Module):
    expansion = 4 # BottleneckBlockでは出力チャネル数が中間層の4倍になる

    def __init__(self, in_channels, out_channels_intermediate, stride=1):
        # out_channels_intermediate はボトルネック部分のチャネル数
        # ブロック全体の出力チャネル数は out_channels_intermediate * self.expansion
        super(BottleneckBlock, self).__init__()
        actual_out_channels = out_channels_intermediate * self.expansion

        self.conv1 = nn.Conv2d(in_channels, out_channels_intermediate, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels_intermediate)
        self.conv2 = nn.Conv2d(out_channels_intermediate, out_channels_intermediate, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels_intermediate)
        self.conv3 = nn.Conv2d(out_channels_intermediate, actual_out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(actual_out_channels)
        # self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != actual_out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, actual_out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(actual_out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)

        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out)) # 最後の畳み込みの後にはReLUなしで、加算後にReLU

        out += identity
        out = F.relu(out)
        return out

In [ ]:
class ResNetCIFAR(nn.Module):
    def __init__(self, block_type, num_blocks_list, num_classes=10):
        # block_type: BasicBlock or BottleneckBlock
        # num_blocks_list: 各ステージのブロック数 (例: ResNet18なら [2,2,2,2])
        super(ResNetCIFAR, self).__init__()
        self.in_channels_current = 64 # 最初の畳み込み層の後のチャネル数

        # CIFAR-10用: 最初の7x7 ConvとMaxPoolはより小さなConvに置き換える
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        # self.relu = nn.ReLU(inplace=True)
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # ImageNet用

        # 残差ブロックのステージ
        # CIFAR-10では、最初のステージのストライドは1 (ダウンサンプリングしない)
        self.layer1 = self._make_layer(block_type, 64, num_blocks_list[0], stride=1)
        self.layer2 = self._make_layer(block_type, 128, num_blocks_list[1], stride=2) # ダウンサンプリング
        self.layer3 = self._make_layer(block_type, 256, num_blocks_list[2], stride=2) # ダウンサンプリング
        self.layer4 = self._make_layer(block_type, 512, num_blocks_list[3], stride=2) # ダウンサンプリング
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # Global Average Pooling
        self.fc = nn.Linear(512 * block_type.expansion, num_classes)

    def _make_layer(self, block_type, out_channels_block, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1) # 最初のブロックのみストライド変更の可能性
        layers = []
        for s in strides:
            layers.append(block_type(self.in_channels_current, out_channels_block, s))
            self.in_channels_current = out_channels_block * block_type.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        # out = self.maxpool(out) # CIFAR-10では初期のMaxPoolを省略することが多い

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

def ResNet18_CIFAR(num_classes=10):
    return ResNetCIFAR(BasicBlock, [2,2,2,2], num_classes)

def ResNet34_CIFAR(num_classes=10):
    return ResNetCIFAR(BasicBlock, [3,4,6,3], num_classes)

def ResNet50_CIFAR(num_classes=10):
    return ResNetCIFAR(BottleneckBlock, [3,4,6,3], num_classes) # BottleneckBlockを使用

In [ ]:
model_resnet_cifar = ResNet18_CIFAR(num_classes=10).to(device)
# model_resnet_cifar = ResNet34_CIFAR(num_classes=10).to(device)
# model_resnet_cifar = ResNet50_CIFAR(num_classes=10).to(device) # より深いモデルは学習に時間がかかる

print("\nResNet-style Model for CIFAR-10 (e.g., ResNet18 variant):\n")
# print(model_resnet_cifar) # 構造が長いためコメントアウト

# ダミー入力でフォワードパスのテスト
dummy_input_res = torch.randn(batch_size_cifar_res // 4, 3, 32, 32).to(device)
try:
    output_dummy_res = model_resnet_cifar(dummy_input_res)
    print("CIFAR Dummy Input Shape (ResNet):", dummy_input_res.shape)
    print("CIFAR Dummy Output Shape (Logits, ResNet):", output_dummy_res.shape)
except Exception as e:
    print("Error during ResNet dummy forward pass:", e)

In [ ]:
# 損失関数とOptimizer
criterion_resnet = nn.CrossEntropyLoss()
optimizer_resnet = optim.SGD(model_resnet_cifar.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
# optimizer_resnet = optim.Adam(model_resnet_cifar.parameters(), lr=0.001)

# 学習率スケジューラ (ResNetの学習でよく使われる)
# 例: 150エポック学習する場合、80, 120エポックで学習率を0.1倍にするなど
scheduler_resnet = torch.optim.lr_scheduler.MultiStepLR(optimizer_resnet, milestones=[80, 120], gamma=0.1)


# 学習ループ
num_epochs_resnet = 100 # ResNetは比較的多くのエポック数が必要な場合がある
print(f"\nResNet-style 学習開始 (CIFAR-10データ、{num_epochs_resnet} epochs)...")

history_resnet = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(num_epochs_resnet):
    model_resnet_cifar.train()
    running_train_loss = 0.0; correct_train = 0; total_train = 0
    start_epoch_time = time.time()
    
    for i, (images, labels) in enumerate(train_loader_cifar_res):
        images, labels = images.to(device), labels.to(device)
        optimizer_resnet.zero_grad()
        outputs = model_resnet_cifar(images)
        loss = criterion_resnet(outputs, labels)
        loss.backward()
        optimizer_resnet.step()
        
        running_train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_train_loss = running_train_loss / total_train
    epoch_train_acc = 100 * correct_train / total_train
    history_resnet['train_loss'].append(epoch_train_loss)
    history_resnet['train_acc'].append(epoch_train_acc)
    
    model_resnet_cifar.eval()
    running_test_loss = 0.0; correct_test = 0; total_test = 0
    with torch.no_grad():
        for images_test, labels_test in test_loader_cifar_res:
            images_test, labels_test = images_test.to(device), labels_test.to(device)
            outputs_test = model_resnet_cifar(images_test)
            loss_test = criterion_resnet(outputs_test, labels_test)
            running_test_loss += loss_test.item() * images_test.size(0)
            _, predicted_test = torch.max(outputs_test.data, 1)
            total_test += labels_test.size(0)
            correct_test += (predicted_test == labels_test).sum().item()
            
    epoch_test_loss = running_test_loss / total_test
    epoch_test_acc = 100 * correct_test / total_test
    history_resnet['test_loss'].append(epoch_test_loss)
    history_resnet['test_acc'].append(epoch_test_acc)
    
    scheduler_resnet.step() # 各エポック終了後にスケジューラを更新
    
    end_epoch_time = time.time()
    epoch_duration = end_epoch_time - start_epoch_time
    
    print(f"Epoch [{epoch+1}/{num_epochs_resnet}] - Dur: {epoch_duration:.1f}s - LR: {optimizer_resnet.param_groups[0]['lr']:.1e} - "
          f"TrL: {epoch_train_loss:.3f}, TrAcc: {epoch_train_acc:.2f}% - "
          f"TeL: {epoch_test_loss:.3f}, TeAcc: {epoch_test_acc:.2f}%")

print("ResNet-style 学習完了!")

In [ ]:
# 学習曲線
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs_resnet + 1), history_resnet['train_loss'], label='Training Loss', marker='.')
plt.plot(range(1, num_epochs_resnet + 1), history_resnet['test_loss'], label='Test Loss', marker='.')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('ResNet Training & Test Loss (CIFAR-10)'); plt.legend(); plt.grid(True)
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs_resnet + 1), history_resnet['train_acc'], label='Training Accuracy', marker='.')
plt.plot(range(1, num_epochs_resnet + 1), history_resnet['test_acc'], label='Test Accuracy', marker='.')
plt.xlabel('Epochs'); plt.ylabel('Accuracy (%)'); plt.title('ResNet Training & Test Accuracy (CIFAR-10)'); plt.legend(); plt.ylim(0,100); plt.grid(True)
plt.tight_layout(); plt.show()

print(f"\n最終テスト精度 (ResNet-style on CIFAR-10): {history_resnet['test_acc'][-1]:.2f}%")

## 5. 考察

*   **ResNetの設計思想のポイント:**
    *   **残差学習とスキップ接続:** 深いネットワークにおける最適化の困難性（Degradation Problem）を、残差関数を学習させ、入力からのスキップ接続で恒等写像を容易に学習できるようにすることで解決しました。これにより、従来よりも遥かに深いネットワークの学習が可能になりました。
    *   **バッチ正規化の活用:** 各畳み込み層の後にバッチ正規化を配置することで、学習を安定させ、勾配の流れを改善し、より大きな学習率の使用を可能にしました。これは深いネットワークの学習に不可欠な要素です。
    *   **ボトルネックデザイン (深いモデル向け):** ResNet-50以上のモデルでは、1x1畳み込みでチャネル数を削減・復元するボトルネック構造を採用し、計算効率を維持しながら深さを追求しました。
    *   **アーキテクチャの単純な繰り返し:** Basic BlockまたはBottleneck Blockという構成要素を繰り返し積み重ねることで、非常に深いネットワークを構築します。

*   **PyTorchによるResNet風モデルの実装 (CIFAR-10用):**
    *   `BasicBlock`（およびオプションで`BottleneckBlock`）を`nn.Module`として定義し、これらを組み合わせて全体のResNetアーキテクチャを構築しました。
    *   CIFAR-10の画像サイズ (32x32) に合わせて、初期の畳み込み層のカーネルサイズやストライド、プーリング層の適用を調整しました。ImageNet用のResNetでは最初の7x7畳み込みとMaxPoolで大幅に空間次元を削減しますが、CIFAR-10ではそこまでの削減は行いません。
    *   ダウンサンプリングは、各ステージの最初のブロックの畳み込み層のストライドを2に設定し、同時にショートカット接続も次元を合わせるために1x1畳み込み（ストライド2）を使用することで行いました。
    *   最後にGlobal Average Poolingと全結合層で分類出力を得ます。

*   **学習結果と課題:**
    *   CIFAR-10においても、ResNetアーキテクチャ（例えばResNet-18相当）は高い性能を発揮することが期待されます。適切な学習率スケジューラやData Augmentationと組み合わせることで、さらに精度が向上します。
    *   論文で示されているように、ResNetは層を深くするほど性能が向上する傾向がありますが（Degradation Problemを解決したため）、ある点を超えると計算コストや過学習のリスクも増大します。データセットの規模や複雑さに応じた適切な深さの選択が重要です。
    *   バッチ正規化は、学習時のバッチサイズに性能が影響を受けることがあります。小さなバッチサイズでは統計量が不安定になる可能性があるため注意が必要です。

ResNetは、その革新的な残差学習のアイデアにより、深層学習におけるネットワークの「深さ」の限界を大きく押し上げました。その後の多くのCNNアーキテクチャは、この残差接続の概念を何らかの形で取り入れています。